# Certificate Characteristics

In [294]:
import numpy as np
import pandas as pd
import os
from pprint import pprint
import json
import time
import random
import re
from collections import defaultdict
import pickle
import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc

In [185]:
everyFile=[]
tmp=os.listdir('/data/data/')
tmp.remove('newbenigndata')
for x in tmp:
    try:
        folder='/data/data/'+x+'/enrichment/cert_info/'
        for y in os.listdir(folder):
            if (len(y) > 10):
                everyFile.append(folder+y)
    except:
        continue

cleanFile=[]
for x in os.listdir('/data/data/newNONTORbenigndata/enrichment/cert_info/'):
        cleanFile.append('/data/data/newNONTORbenigndata/enrichment/cert_info/'+x)


In [186]:
random.Random(28734).shuffle(everyFile)
everyFile=everyFile[0:len(cleanFile)]+cleanFile
print(len(everyFile))

2940


In [187]:
df_cols=['benign', 'indicator', 'error_type', 'cert_validation', 'success', 
         'authorityInfoAccess', 'basicConstraints', 'certificatePolicies',
         'has_expired', 'notAfter', 'notBefore']

col_check=set(df_cols)
error_types=defaultdict(int)
validation_types=defaultdict(int)
usage_types=defaultdict(int)
issuers=defaultdict(int)
ctr=0
random.Random(28734).shuffle(everyFile)
lamow=[]
for x in everyFile:
    if (ctr == 200):
        break
        
    try:
        book=json.load(open(x))['data']
        ctr+=1
        for cert_dict in book:
            df=pd.DataFrame(columns=df_cols)
            df['benign']=[len(x)-x.rfind('/')<11]
            infoString=None  
            if (cert_dict.get('cert_info') == None):
                infoString='certificate_info'
            else:
                infoString='cert_info'
                    
            if (type(cert_dict[infoString]) != dict):
                    df['indicator']=[cert_dict['indicator']]
                    df['success']=[cert_dict['success']]
                    #try:
                        #df['error_type']=re.findall("\[.*\]",cert_dict[infoString])[0]
                    #except Exception as s:
                    df['error_type']=[cert_dict[infoString]]
                    
                    error_types[df['error_type'][0]]+=1

            else:
                df['indicator']=[cert_dict['indicator']]
                df['success']=[cert_dict['success']]
                df['cert_validation'] = [cert_dict.get('cert_validation')]
                
                #try:
                    #validation_types[re.findall("\[.*\]",cert_dict.get('cert_validation'))[0]]+=1
                #except:
                validation_types[df['cert_validation'][0]]+=1
                    
                if (cert_dict[infoString].get('subject') != None):
                    alpha=cert_dict[infoString]['subject'].get('CN',' ')+cert_dict[infoString].get('subjectAltName','')
                    df['subject_match'] = [alpha.find(cert_dict['indicator']) != -1]
                    df['alt_subjects'] = [alpha.count(',')]
                    
                df['issuer']=cert_dict[infoString].get('issuer',{}).get('CN')
                issuers[df['issuer'][0] or '']+=1
                df['keyUsage']=[cert_dict[infoString].get('keyUsage','').split(',')]
                df['extendedKeyUsage']=[cert_dict[infoString].get('extendedKeyUsage','').split(',')]
                for x in df['keyUsage'][0]:
                    usage_types[x]+=1
                
                for q in cert_dict[infoString]:
                    if (q in col_check):
                        df[q]=[cert_dict[infoString][q]]
                        
            lamow.append(df)
    except Exception as e:
        print(e)
        continue
        

'str' object has no attribute 'get'
'str' object has no attribute 'get'


In [188]:
df=pd.concat(lamow,axis=0)

def convertToDt(date_string):
    try:
        if (type(date_string) != float):
            return datetime.datetime.strptime(date_string,'%Y/%m/%d %H:%M.%S')
        else:
            return None
    except Exception as e:
        print(date_string)
        print(e)
        
df['notAfter']=df['notAfter'].apply(convertToDt)
df['notBefore']=df['notBefore'].apply(convertToDt)
df.reset_index(drop=True,inplace=True)

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [195]:
df=pd.concat([df[df.benign == False].loc[0:1500],df[df.benign == True]])

,alt_subjects,authorityInfoAccess,basicConstraints,benign,cert_validation,certificatePolicies,error_type,extendedKeyUsage,has_expired,indicator,issuer,keyUsage,notAfter,notBefore,subject_match,success
0,1.0,OCSP - URI:http://ocsp.int-x3.letsencrypt.org\...,CA:FALSE,False,[SSL: CERTIFICATE_VERIFY_FAILED] certificate v...,Policy: 2.23.140.1.2.1\nPolicy: 1.3.6.1.4.1.44...,NaN,"[TLS Web Server Authentication, TLS Web Clien...",False,arkeisteal.ru.s14.hhos.ru,Let's Encrypt Authority X3,"[Digital Signature, Key Encipherment]",2019-07-24 17:35:41,2019-04-25 17:35:41,False,True
1,NaN,NaN,NaN,False,NaN,NaN,[Errno 110] Connection timed out,NaN,NaN,l0rdix.ddns.net,NaN,NaN,NaT,NaT,NaN,False
2,NaN,NaN,NaN,False,NaN,NaN,[Errno 111] Connection refused,NaN,NaN,177.200.252.229,NaN,NaN,NaT,NaT,NaN,False
3,NaN,NaN,NaN,False,NaN,NaN,[Errno 111] Connection refused,NaN,NaN,187.26.159.211,NaN,NaN,NaT,NaT,NaN,False
4,NaN,NaN,NaN,False,NaN,NaN,[Errno 111] Connection refused,NaN,NaN,177.53.3.110,NaN,NaN,NaT,NaT,NaN,False
5,1.0,OCSP - URI:http://gp.symcd.com\nCA Issuers - U...,CA:FALSE,False,[SSL: CERTIFICATE_VERIFY_FAILED] certificate v...,Policy: 2.23.140.1.2.1\n CPS: https://www.rap...,NaN,"[TLS Web Server Authentication, TLS Web Clien...",True,179.124.9.134,RapidSSL SHA256 CA,"[Digital Signature, Key Encipherment]",2019-04-14 23:59:59,2016-06-20 00:00:00,False,True
6,NaN,NaN,NaN,False,NaN,NaN,[Errno 111] Connection refused,NaN,NaN,200.201.3.56,NaN,NaN,NaT,NaT,NaN,False
7,NaN,NaN,NaN,False,NaN,NaN,[Errno 111] Connection refused,NaN,NaN,200.178.254.133,NaN,NaN,NaT,NaT,NaN,False
8,NaN,NaN,NaN,False,NaN,NaN,[Errno 111] Connection refused,NaN,NaN,179.126.82.156,NaN,NaN,NaT,NaT,NaN,False
9,NaN,NaN,NaN,False,NaN,NaN,[Errno 111] Connection refused,NaN,NaN,201.55.31.206,NaN,NaN,NaT,NaT,NaN,False


In [196]:
df.reset_index(drop=True,inplace=True)

In [203]:
def replaceCertVal(val):
    if (type(val) == str):
        return 1
    elif (val == True):
        return 2
    elif (type(val) == float):
        return 3
          
df['cert_validation']=df['cert_validation'].apply(replaceCertVal)

In [212]:
general_issuers=list(general_issuers.keys())

def replaceIssuer(val):
    if (type(val) == float):
        return None
    elif (type(val) == str):
        try:
            return general_issuers.index(val)
        except:
            return len(general_issuers)

df['issuer']=df['issuer'].apply(replaceIssuer)

In [225]:
def replaceMatch(val):
    if (type(val) == bool):
        return val+1
    return 0
    
df['subject_match']=df['subject_match'].apply(replaceMatch)
df['has_expired']=df['has_expired'].apply(replaceMatch)

In [228]:
df[['alt_subjects','cert_validation','has_expired','issuer','subject_match','success']]

,alt_subjects,cert_validation,has_expired,issuer,subject_match,success
0,1.0,1.0,1,55.0,1,True
1,0.0,3.0,0,0.0,0,False
2,0.0,3.0,0,0.0,0,False
3,0.0,3.0,0,0.0,0,False
4,0.0,3.0,0,0.0,0,False
5,1.0,1.0,2,55.0,1,True
6,0.0,3.0,0,0.0,0,False
7,0.0,3.0,0,0.0,0,False
8,0.0,3.0,0,0.0,0,False
9,0.0,3.0,0,0.0,0,False


In [120]:
general_issuers=defaultdict(int)

for x in issuers:
    if issuers[x] != 1:
        general_issuers[x.split(' ')[0]]+= issuers[x]
        
pprint(general_issuers)

defaultdict(<class 'int'>,
            {'': 46,
             '*': 6,
             '192.168.168.168': 3,
             '1Sizeretioflyb.Triot5sqursh.place': 2,
             'ASA': 2,
             'AlphaSSL': 14,
             'Amazon': 46,
             'COMODO': 160,
             'Certum': 3,
             'Certyfikat': 3,
             'CloudFlare': 49,
             'Daniel': 2,
             'DigiCert': 60,
             'Encryption': 16,
             'Entrust': 4,
             'Gandi': 2,
             'GeoTrust': 13,
             'GlobalSign': 25,
             'Go': 39,
             'Google': 6,
             'HydrantID': 2,
             "Let's": 402,
             'OpenVPN': 2,
             'Parallels': 8,
             'Plesk': 8,
             'PowerLink': 2,
             'QNAP': 5,
             'RDPWayan.wayan.mx': 2,
             'RapidSSL': 86,
             'SANGFOR': 9,
             'Sectigo': 27,
             'SecureCore': 3,
             'Starfield': 9,
             'TRAEFIK': 3,
     

In [121]:
pprint(issuers)

defaultdict(<class 'int'>,
            {'': 46,
             '*': 6,
             '*.domoticz.com': 1,
             '*.empis.pt': 1,
             '*.from.sh': 1,
             '*.sms.org': 1,
             '00:16:b6:8d:c1:aa': 1,
             '10.0.0.218': 1,
             '10.254.254.253': 1,
             '131.221.160.252': 1,
             '153.126.128.94': 1,
             '192.168.168.168': 3,
             '192.168.2.1': 1,
             '1Sizeretioflyb.Triot5sqursh.place': 2,
             '5.135.178.184': 1,
             '5Brwbetjalje.crown': 1,
             '70:e4:22:95:1c:a4': 1,
             '8700.hikvision.com': 1,
             'ASA Temporary Self Signed Certificate': 2,
             'Actalis Organization Validated Server CA G1': 1,
             'Actalis Organization Validated Server CA G2': 1,
             'AlphaSSL CA - SHA256 - G2': 14,
             'Amazon': 46,
             'B31s': 1,
             'Bitbug SGC CA 1': 1,
             'Bitrix': 1,
             'CA': 1,
           

In [ ]:
with open('cert_info.pickle', 'wb') as output:  # Overwrites any existing file.
    pickle.dump(df, output, pickle.HIGHEST_PROTOCOL)
    
with open('error_types.pickle', 'wb') as output:  # Overwrites any existing file.
    pickle.dump(error_types, output, pickle.HIGHEST_PROTOCOL)
    
with open('validation_types.pickle', 'wb') as output:  # Overwrites any existing file.
    pickle.dump(validation_types, output, pickle.HIGHEST_PROTOCOL)
    
with open('usage_types.pickle', 'wb') as output:  # Overwrites any existing file.
    pickle.dump(usage_types, output, pickle.HIGHEST_PROTOCOL)

## Duration of validity as feature, extended certificate

In [289]:
df=df.sample(frac=1).reset_index(drop=True)

In [290]:
X_t, X_v, y_t, y_v = train_test_split(df[['alt_subjects','cert_validation','has_expired','issuer','subject_match','success' ]],df['benign'],test_size=.25,random_state=35)

In [291]:
clf = RandomForestClassifier(n_estimators=100).fit(X_t,y_t)

In [300]:
clf.score(X_t,y_t)

0.6500290191526408

In [284]:
df=pd.concat([df[df.benign == False][0:len(df[df.benign == True])],df[df.benign == True]], axis =0)

In [301]:
print("Confusion matrix:\n%s" % confusion_matrix(y_t, clf.predict(X_t)))

Confusion matrix:
[[332 529]
 [ 74 788]]
